In [128]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display
import cv2
from PIL import Image, ImageDraw
import time

In [129]:
# ONLY ONE OF THE NEXT TWO CELLS NEEDS RUN

In [130]:
# To load image place image file in same directory as notebook and add to dictionary below

Images = {"PIT":"pittscript.png","TREE":"tree.png", "CAT":"cat.jpg", "EXP":"exp.png"}

img = Image.open(Images["TREE"])

width, height = img.size
img = np.array(img)

img = img[:,:,1]/255

In [131]:
# To draw image run this cell and follow these instructions:
#     Left click to draw circles
#     Press escaped when finished

width, height = 800,800 # larger values will take longer to animate
circle_size = [40,30,20] # keep in mind the PDEs use a 3x3 grid. Sizes smaller than 15 die out quickly
circle_size_index = 0

background_color_value = 1 
circle_color = 255

# mouse callback function
def draw_circle(event,x,y,flags,param):
    global circle_size_index
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),circle_size[circle_size_index],circle_color, -1)
        circle_size_index = (circle_size_index + 1) % len(circle_size)
        
# make initial image and create windows/events
img = np.ones((height,width,3), np.uint8)*background_color_value
cv2.namedWindow('image')
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    if cv2.waitKey(20) & 0xFF == 27:
        break
cv2.destroyAllWindows()


img = Image.fromarray(img)

width, height = img.size
img = np.array(img)

img = img[:,:,1]

In [132]:
# This block initializes the concentration arrays

# plot img if desired
#plt.imshow(img)
#plt.show()

# get image dimensions
M, N = img.shape

# initialize concentation arrays
A = np.ones((M,N), dtype=float)*img
B = np.ones((M,N), dtype=float)*img
C = np.ones((M,N), dtype=float)*img
A0 = A.copy()
B0 = B.copy()
C0 = C.copy()

T = np.array([A,B,C])

In [133]:
# This block defines the timestep function used during the animation

# rate constants
a, b, c = .7, 1, 1

def timestep(T):
    
    # copy current concentration arrays (old = current)
    A0 = T[0].copy()
    B0 = T[1].copy()
    C0 = T[2].copy()
    
    # perform jacobi iterations for each chemical
    for T0 in (A0,B0,C0):
        left = np.roll(T0, 1, axis=1)
        right = np.roll(T0, -1, axis=1)
        below = np.roll(T0, -1, axis=0)
        above = np.roll(T0, 1, axis=0)
        
        #T0[1:-1, 1:-1] = 0.25*(left[1:-1, 1:-1] + right[1:-1, 1:-1] +
        #                  above[1:-1, 1:-1] + below[1:-1, 1:-1])
        
        # alternatively use a Moore neighborhood (all 9 pts in 3x3 stencil) for better looking high-res animations
        above_left = np.roll(above, 1, axis=1)
        above_right = np.roll(above,-1, axis=1)
        below_left = np.roll(below, 1, axis=1)
        below_right = np.roll(below, -1, axis=1)
                              
        T0[1:-1, 1:-1] = (1/9)*(left[1:-1, 1:-1] + right[1:-1, 1:-1] +
                          above[1:-1, 1:-1] + below[1:-1, 1:-1] +
                          above_left[1:-1, 1:-1] + above_right[1:-1, 1:-1] +
                          below_left[1:-1, 1:-1] + below_right[1:-1, 1:-1] + T0[1:-1, 1:-1])
        
        # The following keeps the PDE from reflecting at the boundaries.
        #T0[0] = T0[1]
        #T0[-1] = T0[-2]
        #T0[:,0] = T0[:,1]
        #T0[:,-1] = T0[:,-2]

    
    # A0, B0, C0 have been jacobi averaged
    
    # reaction PDEs
    T[0] = A0 + A0*(a*B0 - c*C0) 
    T[1] = B0 + B0*(b*C0 - a*A0)
    T[2] = C0 + C0*(c*A0 - b*B0)

    
    # make sure concentrations are between 0 and 100%
    T[:][T<0] = 0
    T[:][T>1] = 1

    return T

In [134]:
# This block creates the animation by calling the timestep() function above

# modify number of frames to change length of video
frames = 500

# create figure
fig = plt.figure(figsize=(width/100, height/100))
im = plt.imshow(T[0], cmap="bone", animated=True) # other nice looking cmaps inculde bone, greys, cool.
plt.axis("off")                                     #   More can be found at 
                                                    # https://matplotlib.org/examples/color/colormaps_reference.html
# get start time if interested in timing
start = time.time()

# define animation function
def animate(i, T):
    T = timestep(T)
    im.set_array(T[0]) # could use T[1] and T[2]
    progress.progress += 1
    return im

# create progress bar
progress = display.ProgressBar(frames)
progress.display()

# process first several iterations to avoid sharp changes in contrast at beginning of video
for _ in range(30):
    timestep(T)
    progress.progress += 1

# create animation
anim = FuncAnimation(fig, animate, frames=frames, interval=40, fargs=(T,), repeat=False)
plt.close()

###### - This block displays in notebook
#
video = anim.to_html5_video()
stop = time.time()
print(stop-start, "s")
display.HTML(video)
#
######

###### - This block saves video -- large file sizes will need to be saved
# 
#save_string = time.strftime("%Y%m%d-%H%M%S") + ".mp4"
#anim.save(save_string, fps=30, dpi=120, savefig_kwargs={'facecolor':'black'})
#print("saved as: ", save_string)
#stop = time.time()
#print(stop-start, "s")
#
######

[===============================================================] 531/500

197.57595443725586 s
